In [1]:
import json
import logging
import requests

In [2]:
def takeResponseAttr(attr_name, resp_text, quote = '"'):
    attr_name = attr_name+'='+quote
    start = resp_text.find(attr_name)+len(attr_name)
    end = resp_text.find(quote, start)
    return resp_text[start: end]

In [3]:
def takeResponseDict(resp_text):
    #возвращает только первый словарь.
    #будут проблемы, если будет вложенный словарь
    start = resp_text.find('{')
    end = resp_text.find('}', start)
    return eval(resp_text[start: end+1])

In [4]:
def takeConvertedDict(source_dict):
    _d={}
    for k, n in source_dict.items():
        n = n.replace('\\/','/')
        if n.count('|') > 2:
            for item in n.split('|'):
                if item.count('=') > 0:
                    i, j = item.split('=')
                else:
                    i = k
                    j = item
                _d[i] = j
        else:
            _d[k] = n
        
    return _d

In [5]:
formattedStrLambda = lambda item_dict: '&'.join(map(lambda x: '{}={}'.format(x[0],x[1]), item_dict.items()))

In [6]:
logging.basicConfig(level=logging.DEBUG)
captcha_link ='https://captcha.kik.com/?id='
captcha_id = '3-CAISeFB-ktoCL82eTHfb1e21_IaYfoSgigifgkPlgml_HsDaWUeprIeOpcT0c1mEY6WJvGRUiZ9ufLsRosdV3L3NNvLPXjx81yknkc43M0MA1LTL4rVLQtUt5cxak2bwqiyjfx774cz3TFNWJx9umPOTeEsQ4i1R6ffEWBoQRnJLIBkuLz4jNRrsmHU53iIfXh2cl7DcCHEUy_Ay4xOXurBcNWMNpO-Pp7SG5zPEryoQq2OV1HzvPduxpEZXP0pYcA'

ses = requests.session()
res = ses.get(captcha_link+captcha_id)

ses.headers.update({'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'})
ses.headers.update({'referer': captcha_link+captcha_id, 'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'\
                   ,'authority':'funcaptcha.com'})

next1 ='https://funcaptcha.com/fc/api/?onload=callbackCaptchaReady'
res1 = ses.get(next1)
#get next link
next2 = takeResponseAttr('t.src',res1.text)
res2 = ses.get(next2)
#get next link
token_dict = {'bda':'W3sia2V5IjoiYXBpX3R5cGUiLCJ2YWx1ZSI6ImpzIn0seyJrZXkiOiJwIiwidmFsdWUiOjF9L'\
                       'Hsia2V5IjoiZiIsInZhbHVlIjoiNzY1MWVkMTE5ZmRmZDhlNWQzMTA0NGYzZjk3N2ZkMDIifSx7Im'\
                       'tleSI6ImNzIiwidmFsdWUiOjF9XQ=='\
                       ,'public_key':'92A09218-4AD3-CC2B-8145-87CD0B851124'\
                       ,'site':'https://captcha.kik.com'\
                       ,'data[kik_id]':captcha_id}
            
next3 =  takeResponseAttr('var S7e', res2.text)\
        +takeResponseAttr('var Z8e', res2.text)\
        +'&'+formattedStrLambda(takeConvertedDict(token_dict))
        
res3 = ses.get(next3)
#токен получен успешно.

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): captcha.kik.com
DEBUG:urllib3.connectionpool:https://captcha.kik.com:443 "GET /?id=3-CAISeFB-ktoCL82eTHfb1e21_IaYfoSgigifgkPlgml_HsDaWUeprIeOpcT0c1mEY6WJvGRUiZ9ufLsRosdV3L3NNvLPXjx81yknkc43M0MA1LTL4rVLQtUt5cxak2bwqiyjfx774cz3TFNWJx9umPOTeEsQ4i1R6ffEWBoQRnJLIBkuLz4jNRrsmHU53iIfXh2cl7DcCHEUy_Ay4xOXurBcNWMNpO-Pp7SG5zPEryoQq2OV1HzvPduxpEZXP0pYcA HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): funcaptcha.com
DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "GET /fc/api/?onload=callbackCaptchaReady HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cdn.funcaptcha.com
DEBUG:urllib3.connectionpool:https://cdn.funcaptcha.com:443 "GET /fc/js/ba1e486f2d32eee8fdf97ee89ec06b14/standard/funcaptcha_api.js HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "GET /fc/gt2/?callback=fcJSONPCallback&bda=W3sia2V5IjoiYXBpX3R5cGUiLCJ2YWx1ZSI6ImpzIn0s

In [7]:
auth_json = takeConvertedDict(takeResponseDict(res3.text))
auth_json['ps'] = captcha_id

next4 = 'https://funcaptcha.com/fc/gc/?'\
        +formattedStrLambda(takeConvertedDict(auth_json))

res4 = ses.get(next4, data = takeConvertedDict(auth_json))

DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "GET /fc/gc/?token=6805a37e1f1aade09.46787726&r=eu-west-1&metabgclr=transparent&metatimeclr=%237a7d8e&meta=6&guitextcolor=%2326beff&metaiconclr=transparent&pk=92A09218-4AD3-CC2B-8145-87CD0B851124&injs=https://cdn.funcaptcha.com/fc/assets/graphics/kik/scripts/kik_help_8.js&at=40&ps=3-CAISeFB-ktoCL82eTHfb1e21_IaYfoSgigifgkPlgml_HsDaWUeprIeOpcT0c1mEY6WJvGRUiZ9ufLsRosdV3L3NNvLPXjx81yknkc43M0MA1LTL4rVLQtUt5cxak2bwqiyjfx774cz3TFNWJx9umPOTeEsQ4i1R6ffEWBoQRnJLIBkuLz4jNRrsmHU53iIfXh2cl7DcCHEUy_Ay4xOXurBcNWMNpO-Pp7SG5zPEryoQq2OV1HzvPduxpEZXP0pYcA&ht=1&cdn_url=https://cdn.funcaptcha.com/fc&surl=https://funcaptcha.com&challenge_url=/fc/api/bootstrap/&challenge_url_cdn=https://cdn.funcaptcha.com/fc/js/ba1e486f2d32eee8fdf97ee89ec06b14/standard/fc_bootstrap.js&noscript=Disable HTTP/1.1" 200 None


In [8]:
ses.headers.update({'referer': next4})

form_data = {'sid': auth_json.get('r')
             ,'session_token':auth_json.get('token')
             ,'render_type':'canvas'
             ,'category':'Site+URL'
             ,'action':captcha_link+captcha_id
             ,'analytics_tier':auth_json.get('at')
            }

next6 = 'https://funcaptcha.com/fc/a/'
res6 = ses.post(next6, data = takeConvertedDict(form_data))

DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "POST /fc/a/ HTTP/1.1" 200 None


In [9]:
print(res6.text)

{"logged":true}


In [10]:
request = {
          "method": "POST",
          "url": "https://funcaptcha.com/fc/ekey/",
          "httpVersion": "http/2.0",
          "headers": [
            {
              "name": "cookie",
              "value": "nlbi_384110=VgNpCsQULDA9HqBzqbeGIAAAAADGtEX4xjGAzCO7ogSWSUm8; visid_incap_384110=jO8w1dckR9i287fkcE4+jU1TKVoAAAAAQUIPAAAAAADmW2MRJizAZ74T2j/ZwaXZ; incap_ses_377_384110=+ruGHKSe30BQf5JThV87BaEhMVoAAAAASJPh4ZnREKFRZH/5uSi2zA==; _ga=GA1.2.1484574070.1512659665; timestamp=151359700521458"
            },
            {
              "name": "origin",
              "value": "https://funcaptcha.com"
            },
            {
              "name": "accept-encoding",
              "value": "gzip, deflate, br"
            },
            {
              "name": "accept-language",
              "value": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7"
            },
            {
              "name": "user-agent",
              "value": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36"
            },
            {
              "name": "x-requested-with",
              "value": "XMLHttpRequest"
            },
            {
              "name": "content-length",
              "value": "92"
            },

            {
              "name": "x-newrelic-timestamp",
              "value": "151359700521458"
            },
            {
              "name": "x-requested-id",
              "value": "zAGeLFGqN1pgGeCfPWwA/RuPhJ/MzBC/"
            },
            {
              "name": "content-type",
              "value": "application/x-www-form-urlencoded; charset=UTF-8"
            },
            {
              "name": "accept",
              "value": "*/*"
            },
            {
              "name": "cache-control",
              "value": "no-cache"
            },
            {
              "name": "referer",
              "value": "https://funcaptcha.com/fc/gc/?token=1785a37aa42237c23.93341525&r=eu-west-1&metabgclr=transparent&metatimeclr=%237a7d8e&meta=6&guitextcolor=%2326beff&metaiconclr=transparent&pk=92A09218-4AD3-CC2B-8145-87CD0B851124&injs=https://cdn.funcaptcha.com/fc/assets/graphics/kik/scripts/kik_help_8.js&at=40&ps=id&ht=1&cdn_url=https://cdn.funcaptcha.com/fc&surl=https://funcaptcha.com&ps=3-CAISeFB-ktoCL82eTHfb1e21_IaYfoSgigifgkPlgml_HsDaWUeprIeOpcT0c1mEY6WJvGRUiZ9ufLsRosdV3L3NNvLPXjx81yknkc43M0MA1LTL4rVLQtUt5cxak2bwqiyjfx774cz3TFNWJx9umPOTeEsQ4i1R6ffEWBoQRnJLIBkuLz4jNRrsmHU53iIfXh2cl7DcCHEUy_Ay4xOXurBcNWMNpO-Pp7SG5zPEryoQq2OV1HzvPduxpEZXP0pYcA"
            }
          ],
          "queryString": [],
          "cookies": [
            {
              "name": "nlbi_384110",
              "value": "VgNpCsQULDA9HqBzqbeGIAAAAADGtEX4xjGAzCO7ogSWSUm8",
              "expires": 'null',
              "httpOnly": 'false',
              "secure": 'false'
            },
            {
              "name": "visid_incap_384110",
              "value": "jO8w1dckR9i287fkcE4+jU1TKVoAAAAAQUIPAAAAAADmW2MRJizAZ74T2j/ZwaXZ",
              "expires": 'null',
              "httpOnly": 'false',
              "secure": 'false'
            },
            {
              "name": "incap_ses_377_384110",
              "value": "+ruGHKSe30BQf5JThV87BaEhMVoAAAAASJPh4ZnREKFRZH/5uSi2zA==",
              "expires": 'null',
              "httpOnly": 'false',
              "secure": 'false'
            },
            {
              "name": "_ga",
              "value": "GA1.2.1484574070.1512659665",
              "expires": 'null',
              "httpOnly": 'false',
              "secure": 'false'
            },
            {
              "name": "timestamp",
              "value": "151359700521458",
              "expires": 'null',
              "httpOnly": 'false',
              "secure": 'false'
            }
          ],
          "headersSize": -1,
          "bodySize": 92,
          "postData": {
            "mimeType": "application/x-www-form-urlencoded; charset=UTF-8",
            "text": "sid=eu-west-1&session_token=1785a37aa42237c23.93341525&game_token=8885a37aa42cf8871.64074950",
            "params": [
              {
                "name": "sid",
                "value": "eu-west-1"
              },
              {
                "name": "session_token",
                "value": "1785a37aa42237c23.93341525"
              },
              {
                "name": "game_token",
                "value": "8885a37aa42cf8871.64074950"
              }
            ]
          }
        }
h = {i['name']:i['value'] for i in request['headers']} 

In [11]:
game_data = {'sid': auth_json.get('r')
            ,'session_token':auth_json.get('token')
            ,'game_token':'8885a37aa42cf8871.64074950'}

#game_token

next7 = 'https://funcaptcha.com/fc/ekey/'
res7 = ses.post(next7, headers=h, data=takeConvertedDict(game_data))
print(res7.text)

DEBUG:urllib3.connectionpool:https://funcaptcha.com:443 "POST /fc/ekey/ HTTP/1.1" 200 None


{"error":"DENIED ACCESS"}
